In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazonreviews/train.ft.txt.bz2
/kaggle/input/amazonreviews/test.ft.txt.bz2


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline


import os
print(os.listdir("../input"))


['amazonreviews']


Data preprocessing

In [3]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('../input/amazonreviews/train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('../input/amazonreviews/test.ft.txt.bz2')

In [4]:
size=10000

train_labels=train_labels[:10000]

train_texts=train_texts[:10000]

test_labels=test_labels[:10000]

test_texts=test_texts[:10000]

In [5]:
n1_train=sum(train_labels)
n0_train=size-n1_train

n1_test=sum(test_labels)
n0_test=size-n1_test


print('n1_train: '+str(n1_train))
print('n0_train: '+str(n0_train))

print('n1_test: '+str(n1_test))
print('n0_test: '+str(n0_test))

n1_train: 4903
n0_train: 5097
n1_test: 5125
n0_test: 4875


In [6]:
#Data cleaning
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(max_features=2000)
train_texts= vec.fit_transform(train_texts)

In [11]:
train_texts=train_texts.toarray()

In [12]:
vec = CountVectorizer(max_features=2000)
test_texts= vec.fit_transform(test_texts).toarray()

Build SVM model

In [13]:
from sklearn.svm import SVC

In [14]:
clf = SVC(kernel = "linear")
clf.fit(train_texts,train_labels)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
#prediction
pred = clf.predict(test_texts)

In [18]:
#Evaluate model
print(roc_auc_score(test_labels, pred))



0.5634896810506567
